# Catching Criminals with Math
### Identification of fraudulent credit card transactions using statistical and machine learning models
#### March 8th, 2019

#### Authors: Angad S. Kalra, Shuang Di, Pulkit Mathur
#### Collaborators: Shobhit Jain and Adam Rahman

### Introduction
(TODO...)

###  Methods

#### Feature Engineering 
* TODO

#### Model Training 
* TODO

#### Model Evaluation 
* TODO

### Results
(TODO...)

### Discussion
(TODO...)

### Limitations
(TODO...)